In [1]:
import os #used for system commands
import tempfile #used to create temporary folders to store data
import zipfile #used to extract tar files
import urllib #used to download data via http
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from datetime import datetime, timedelta #used to manipulate time :)
from glob import glob #used for manipulating pathnames

import numpy as np
from matplotlib import pyplot as plt
import pyart
from IPython.display import HTML, Image, clear_output
import cartopy.crs as ccrs # A toolkit for map projections
import ipywidgets as widgets
from ipywidgets import Layout


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Function which generates a plot for each sweep 
def fast_plot(odim_ffn, cdict, img_path): 

    #open figure 
    fig = plt.figure(figsize=(10, 8)) 

    #load radar object 
    my_radar = pyart.aux_io.read_odim_h5(odim_ffn, file_field_names=True)

    #find limits 
    radar_lat = my_radar.latitude['data'][0]
    radar_lon = my_radar.longitude['data'][0]
    min_lat   = radar_lat - cdict['prng']
    max_lat   = radar_lat + cdict['prng'] 
    min_lon   = radar_lon - cdict['prng'] 
    max_lon   = radar_lon + cdict['prng'] 

    # Set up the GIS projection 
    projection = ccrs.Mercator( 
                    central_longitude=radar_lon, 
                    min_latitude=min_lat, max_latitude=max_lat) 

    #load diplay class for radar using cartopy 
    display = pyart.graph.RadarMapDisplay(my_radar) 

    #load cmaps 
    if cdict['cmap']=='HomeyerRainbow': 
        cmap = pyart.graph.cm_colorblind.HomeyerRainbow 
    elif cdict['cmap']=='NWSRef': 
       cmap = 'pyart_NWSRef' 

   #create plot 
    display.plot_ppi_map(cdict['field'], cdict['tilt'], 
                           projection=projection, colorbar_flag=True, 
                           min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat, 
                           vmin=cdict['vmin'], vmax=cdict['vmax'], cmap=cmap, 
                           resolution='10m') 

#     #here is our pretty colorbar code 
#     lb = display._get_colorbar_label(field) 
#     cb = plt.colorbar(display.plots[0]) 
#     cb.ax.tick_params(labelsize=12) 
#     cb.set_label(lb, fontsize=12) 

   #Now we add lat lon lines 
    gl = display.ax.gridlines(draw_labels=True, 
                             linewidth=2, color='gray', alpha=0.5, 
                             linestyle='--') 

    gl.xlabel_style = {'size': 12} 
    gl.ylabel_style = {'size': 12} 

    gl.xlabels_top = False 
    gl.ylabels_right = False 

    out_ffn = img_path + '/' + os.path.basename(odim_ffn)[:-3] + '.png' 

    plt.savefig(out_ffn, dpi=100)  # Saving figure. 
    plt.close()  # Release memory 
    fig.clf()  # Clear figure 


def build_animation(cdict): 

    #parse inputs 
    radar_id_str = str(cdict['rid']).zfill(2) #convert radar id to a string and fill with a leading 0 if only one digit 
    date_str     = cdict['rdate'].strftime('%Y/%m/%d') 
    start_dt     = datetime.strptime(date_str + ' ' + cdict['rtime'], '%Y/%m/%d %H:%M') 
    end_dt       = start_dt + timedelta(minutes = cdict['rdur']) 

    #build request filename url 
    zip_fn       = radar_id_str + '_' + cdict['rdate'].strftime('%Y%m%d') + '.pvol.zip' 
    request_url  = '/'.join([base_url, 'odim_pvol', radar_id_str, cdict['rdate'].strftime('%Y'), 'vol', zip_fn]) 

    #download the zip file 
    if not os.path.isfile(zip_fn): 
        print('Fetching:', request_url) 
        urllib.request.urlretrieve(request_url, zip_fn) 

    #extract the zip file to a temporary directory 
    temp_dir = tempfile.mkdtemp() 
    zip_fh = zipfile.ZipFile(zip_fn) 
    zip_fh.extractall(path = temp_dir) 
    zip_fh.close() 
    #os.system('rm ' + zip_fn) 

    #list all the volumes extracted from the zip file 
    file_list = sorted(glob(temp_dir + '/*')) 

    #now let's read the datetime numbers of all the volumes for comparision 
    file_dt_list = [] 
    for i, fname in enumerate(file_list): 
        file_dt_list.append(datetime.strptime(os.path.basename(fname)[3:18],'%Y%m%d_%H%M%S')) 

    #find the index of volumes within our start and end times 
    file_dt_array    = np.array(file_dt_list) 
    filter_file_list = [] 
    for i, file_dt in enumerate(file_dt_array): 
        if file_dt >= start_dt and file_dt <= end_dt: 
            filter_file_list.append(file_list[i]) 

    
    img_path = tempfile.mkdtemp() 

    #generate images 
    img_path = tempfile.mkdtemp() 
    n_img = len(filter_file_list) 
    process_bar = widgets.IntProgress( 
                   value=1, 
                   min=1, 
                   max=n_img, 
                   step=1, 
                   description='Rendering:', 
                   orientation='horizontal', 
                   readout=False, 
                   readout_format='d' 
               ) 
    display(process_bar) 
    for odim_ffn in filter_file_list: 
        fast_plot(odim_ffn, cdict, img_path) 
        process_bar.value += 1 # signal to increment the progress bar 

    return img_path

In [3]:
warnings.filterwarnings('ignore')  
warnings.simplefilter('ignore')  
  
#specific settings  
base_url     = 'http://dapds00.nci.org.au/thredds/fileServer/rq0' #base url for NCI dataset  
  
def init_ui():
    global wrid, wdate, wtime, wdur, wfld, wtilt, wvmin, wvmax, wprng, wcmap
    
    wrid = widgets.Dropdown(
        options=[('03: Wollongong',1), ('28: Grafton',2), ('50: Marburg',3)],
        value=3,
        description='Radar Site:',
        disabled=False,
    )
    wdate = widgets.DatePicker(  
        value=datetime.strptime('2008-11-16','%Y-%m-%d'),  
        description='Date (UTC):',  
        disabled=False  
    )  
    wtime = widgets.Text(  
        value='05:00',  
        description='Time (UTC):',  
        disabled=False  
    )  
    wdur = widgets.Dropdown(  
        options=[('60',1), ('120',2), ('180',3)],  
        value=1,  
        description='Period (min):',  
        disabled=False,  
    )  
    wfld = widgets.Dropdown(  
        options=[('DBZH',1), ('VRADH',2)],  
        value=1,  
        description='Field:',  
        disabled=False,  
    )
    wtilt = widgets.IntSlider(
        value=2,
        min=1,
        max=14,
        step=1,
        description='Tilt:',
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    wvmin = widgets.Text(
        value='-10',
        description='Min. Value:',
        disabled=False
    )
    wvmax = widgets.Text(
        value='65',
        description='Max. Value:',
        disabled=False
    )
    wprng = widgets.Text(
        value='1',
        description='Range (deg.)',
        disabled=False
    )
    wcmap = widgets.Dropdown(
        options=[('HomeyerRainbow',1), ('NWSRef',2)],
        value=1,
        description='Colormap:',
        disabled=False,
    )
  
    title = widgets.HTML(
        value="<font size=4>Configuration</font>",
        description='',
    )  
  
    space = widgets.HTML(
        value="<font size=4>Process</font>",
        description='',
    )  
  
    button = widgets.Button(
        description='Generate Images',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check'
    )
  
    #set up display
    left_box = widgets.VBox([wdate, wtime, wdur])
    middle_box = widgets.VBox([wrid, wfld, wtilt])
    right_box = widgets.VBox([wvmin, wvmax, wprng, wcmap])
  
    #assign button function to button event  
    button.on_click(on_button_clicked)
  
    #show widgets in V Box  
    ui = widgets.VBox([title, widgets.HBox([left_box, middle_box, right_box]), space, button])
    display(ui)
      
def load_image(k):
    global img_path, img_list

    img_widget = Image(open(os.path.join(img_path,img_list[k-1]),'rb').read())
    display(img_widget)
  
#define button function
def on_button_clicked(b):  
    global img_path, img_list  
    global wrid, wdate, wtime, wdur, wfld, wtilt, wvmin, wvmax, wprng, wcmap
    
    #extract vars  
    rid   = int(wrid.options[wrid.value-1][0][0:2])  
    rdate = wdate.value  
    rtime = wtime.value  
    rdur  = int(wdur.options[wdur.value-1][0])  
    tilt  = wtilt.value - 1 #convert to python index  
    field = wfld.options[wfld.value-1][0]  
    vmin  = float(wvmin.value)  
    vmax  = float(wvmax.value)  
    prng  = float(wprng.value)  
    cmap  = wcmap.options[wcmap.value-1][0]
    #build dictionary  
    cdict = {'rid':rid, 'rdate':rdate, 'rtime':rtime, 'rdur':rdur, 'tilt':tilt, 'field':field, 'vmin':vmin, 'vmax':vmax, 'prng':prng, 'cmap':cmap}  
    
    #reset display  
    clear_output()  
    init_ui()  
    
    #call function  
    img_path = build_animation(cdict)  
    #now display  
    #list images  
    img_list = list(filter(lambda fn:fn.lower().endswith('.png'), sorted(os.listdir(img_path))))  
    n_img = len(img_list)  
  
    play = widgets.Play(  
        interval=500,  
        value=1,  
        min=1,  
        max=n_img,  
        step=1,  
        description="Press play",  
        disabled=False  
        )  
    slider = widgets.IntSlider(value=1,  
                           min=1,   
                           max=n_img,   
                           description='Step:',  
                           readout=True)  
    widgets.jslink((play, 'value'), (slider, 'value'))  
    img_ui = widgets.HBox([play, slider])  
    img_ui.layout.align_items = 'stretch'  
    image_frame = widgets.interactive_output(load_image, {'k': slider});  
    display(img_ui, image_frame)  
  
#init ui  
init_ui()

IntProgress(value=1, description='Rendering:', max=5, min=1)

Output()